In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [3]:
#  Import and read the charity_data.csv.
import pandas as pd 
motor_df = pd.read_csv('Resources/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv')
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
0,2018,15232120,VAN TRUCK,AGRICULTURAL TRUCK,Parked,Not Entered,North,Gas,2015.0,NY,0.0,4.0,NISSA,HUMAN,Not Entered,HUMAN,Not Entered,Not Entered,3N6CM0KN2FK713460
1,2018,15232535,SUBURBAN,PASSENGER OR SUBURBAN,Making U Turn,Not Entered,South,Gas,2015.0,NY,1.0,6.0,HONDA,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,5J6TF2H55FL001556
2,2018,15232536,SEDAN,Not Entered,Going Straight Ahead,Not Entered,North,Not Entered,NaN,PA,1.0,NaN,NaN,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,NaN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
4,2018,15232556,SUBURBAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2009.0,NY,2.0,8.0,CADIL,HUMAN,Not Applicable,HUMAN,Not Applicable,Not Entered,3GYFK22209G213337


# Cleaning the Data

In [4]:
# Drop all NA from rows

motor_df = motor_df.dropna()

In [5]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not Entered
# Drop these rows

not_entered = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Entered'].index
motor_df.drop(not_entered, inplace = True)

In [6]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not applicable
# Drop these rows

not_app = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [7]:
# Get Names for index for which Contributing Factor 1 Description is Unknown
# Drop these rows

unknown = motor_df[motor_df['Contributing Factor 1 Description'] == 'Unknown'].index
motor_df.drop(unknown, inplace = True)

In [8]:
# Get unknown values in direction of travel
# Drop these rows

unknown2 = motor_df[motor_df['Direction of Travel'] == 'Unknown'].index
motor_df.drop(unknown2, inplace = True)

In [9]:
# Get Not Applicable in Direction of Travel and drop these rows

not_app = motor_df[motor_df['Direction of Travel'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [10]:
# Looking at the Vehicle Make column
motor_count = motor_df['Vehicle Make'].value_counts()

# Determine which values to replace if counts are less than 5000
replace_Vehicle = list(motor_count[motor_count < 3000].index)
replace_Vehicle

# Replace in dataframe
for car in replace_Vehicle:
    motor_df["Vehicle Make"] = motor_df["Vehicle Make"].replace(car,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Make'].value_counts()

TOYOT    57239
HONDA    46095
CHEVR    44133
FORD     44126
NISSA    35334
Other    33098
JEEP     21089
HYUND    18128
DODGE    16350
SUBAR    13317
KIA       9343
GMC       8263
BMW       8175
ME/BE     7529
VOLKS     7428
CHRYS     7369
LEXUS     5752
ACURA     5745
MAZDA     5685
RAM       4640
BUICK     4562
INFIN     4461
MITSU     3586
AUDI      3495
CADIL     3133
Name: Vehicle Make, dtype: int64

In [11]:
# Recode Vehicle Make to integer: LinearRegression does not take string

def Classification(X):
    if X == 'TOYOT':
        return 0
    elif X == 'HONDA':
        return 1
    elif X == 'CHEVR':
        return 2
    elif X == 'FORD':
        return 3
    elif X == 'NISSA':
        return 4
    elif X == 'Other':
        return 5
    elif X == 'JEEP':
        return 6
    elif X == 'HYUND':
        return 7
    elif X == 'DODGE':
        return 8
    elif X == 'SUBAR':
        return 9
    elif X == 'KIA':
        return 10
    elif X == 'GMC':
        return 11
    elif X == 'BMW':
        return 12
    elif X == 'ME/BE':
        return 13
    elif X == 'VOLKS':
        return 14
    elif X == 'CHRYS':
        return 15
    elif X == 'LEXUS':
        return 16
    elif X == 'ACURA':
        return 17
    elif X == 'MAZDA':
        return 18
    elif X == 'RAM':
        return 19
    elif X == 'BUICK':
        return 20
    elif X == 'INFIN':
        return 21
    elif X == 'MITSU':
        return 22
    elif X == 'AUDI':
        return 23
    else :
        return 24

motor_df['Vehicle Make'] = motor_df['Vehicle Make'].apply(Classification)

In [12]:
# Look at Regristration Class value counts for binning
class_count = motor_df['Registration Class'].value_counts()


# Determine which values to replace if counts are less than 4000
replace_class = list(class_count[class_count < 2000].index)

# Replace in dataframe
for Class in replace_class:
    motor_df['Registration Class'] = motor_df['Registration Class'].replace(Class,"Other")
    
# Check to make sure binning was successful
motor_df['Registration Class'].value_counts()

PASSENGER OR SUBURBAN         347712
OMNIBUS - TAXI                 20924
AGRICULTURAL TRUCK             14251
Other                          10599
COMMERCIAL                      9818
SPECIAL PASSENGER               5611
POLITICAL SUBDIVISION           4352
MOTORCYCLE                      2771
INTERNATIONAL REGISTRATION      2037
Name: Registration Class, dtype: int64

In [13]:
# Action Prior to accident
actions = motor_df['Action Prior to Accident'].value_counts()

# Determine which values to replace if counts are less than 4000
replace_action = list(actions[actions < 2000].index)

# Replace in dataframe
for action in replace_action:
    motor_df['Action Prior to Accident'] = motor_df['Action Prior to Accident'].replace(action,"Other")
    
# Check to make sure binning was successful
motor_df['Action Prior to Accident'].value_counts()

Going Straight Ahead        243029
Making Left Turn             46153
Backing                      23642
Slowing or Stopping          22816
Making Right Turn            21186
Changing Lanes               20355
Starting in Traffic           7221
Merging                       5984
Other                         5926
Starting from Parking         5644
Overtaking/Passing            4366
Making U Turn                 4236
Entering Parked Position      3154
Stopped in Traffic            2309
Parked                        2054
Name: Action Prior to Accident, dtype: int64

In [14]:
# Look at Vehicle Body Type value counts for binning
body_types = motor_df['Vehicle Body Type'].value_counts()


# Determine which values to replace if counts are less than 4000
replace_body = list(body_types[body_types < 1000].index)

# Replace in dataframe
for body_type in replace_body:
    motor_df['Vehicle Body Type'] = motor_df['Vehicle Body Type'].replace(body_type,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Body Type'].value_counts()

SUBURBAN          168435
4 DOOR SEDAN      168355
PICKUP TRUCK       35355
2 DOOR SEDAN       13394
VAN TRUCK           9017
TAXI                3877
BUS (OMNIBUS)       3346
Other               3202
DUMP                2962
MOTORCYCLE          2812
CONVERTIBLE         2332
TRACTOR             2059
DELIVERY TRUCK      1790
UTILITY             1139
Name: Vehicle Body Type, dtype: int64

In [47]:
def Classification(X):
    if X == 'SUBURBAN':
        return 0
    elif X == '4 DOOR SEDAN':
        return 1
    elif X == 'PICKUP TRUCK':
        return 2
    elif X == '2 DOOR SEDAN':
        return 3
    elif X == 'VAN TRUCK':
        return 4
    elif X == 'TAXI':
        return 5
    elif X == 'BUS (OMNIBUS)':
        return 6
    elif X == 'Other':
        return 7
    elif X == 'DUMP':
        return 8
    elif X == 'MOTORCYCLE':
        return 9
    elif X == 'CONVERTIBLE':
        return 10
    elif X == 'TRACTOR':
        return 11
    elif X == 'DELIVERY TRUCK':
        return 12
    else:
        return 13
    
motor_df['Vehicle Body Type'] = motor_df['Vehicle Body Type'].apply(Classification)    

In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description'
# and 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 2','Contributing Factor 2 Description'], axis=1)

In [16]:
# Create our target
y = motor_df['Vehicle Make'] 

# Create our features
X = motor_df.drop(columns = 'Vehicle Make')
X = pd.get_dummies(X)

In [19]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, criterion = 'entropy', random_state=78) 

In [22]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [23]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [28]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["TOYOTA", "HONDA", "CHEVR", "FORD", "NISSAN", "Other", "JEEP", "HYUNDAI", "DODGE",
               "SUBARU", "KIA", "GMC", "BMW", "MERCEDES", "VOLKSWAGEN", "CHRYSLER", "LEXUS",
               "ACURA", "MAZDA", "RAM", "BUICK", "INFINIY", "MITSUBISHI", "AUDI", "CADILLAC"])

cm_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
TOYOTA,5033,2259,1134,857,1238,329,756,551,315,378,...,144,96,117,107,67,74,82,54,49,18
HONDA,2487,2733,850,638,1222,370,686,554,219,548,...,113,78,84,112,12,69,72,70,49,19
CHEVR,1124,1092,2642,1757,758,363,501,358,455,308,...,113,109,82,52,293,115,76,22,35,53
FORD,1189,901,1811,3266,659,429,692,223,428,225,...,75,85,62,50,174,55,53,43,24,54
NISSAN,1831,1540,816,677,1208,218,575,420,152,363,...,84,90,83,80,29,65,77,33,44,28
Other,607,605,517,501,266,4429,281,116,118,89,...,57,85,40,25,10,78,48,32,27,62
JEEP,560,490,409,533,443,113,1449,63,236,287,...,99,66,61,34,6,34,34,24,14,25
HYUNDAI,937,982,437,263,529,110,165,358,67,136,...,31,21,34,38,1,28,21,19,21,12
DODGE,438,333,667,593,220,153,457,86,596,31,...,75,53,47,15,3,15,46,4,11,25
SUBARU,451,694,293,209,374,41,212,115,23,541,...,30,3,7,44,0,7,2,21,11,3


In [40]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score_df = pd.DataFrame(
    acc_score, index=["TOYOTA", "HONDA", "CHEVR", "FORD", "NISSAN", "Other", "JEEP", "HYUNDAI", "DODGE","SUBARU", "KIA", 
                      "GMC", "BMW", "MERCEDES", "VOLKSWAGEN", "CHRYSLER", "LEXUS", "ACURA", "MAZDA", "RAM", "BUICK", 
                      "INFINIY", "MITSUBISHI", "AUDI", "CADILLAC"], columns=["precision", "recall", "f1-score", "support"])

In [45]:
# Displaying results
print("Confusion Matrix")
#display(cm_df)
print(f"Accuracy : {acc_score}")
print("Classification Report")
#print(classification_report(y_test, predictions))
acc_score_df

Confusion Matrix
Accuracy : 0.23052268008687415
Classification Report


,precision,recall,f1-score,support
TOYOTA,0.230523,0.230523,0.230523,0.230523
HONDA,0.230523,0.230523,0.230523,0.230523
CHEVR,0.230523,0.230523,0.230523,0.230523
FORD,0.230523,0.230523,0.230523,0.230523
NISSAN,0.230523,0.230523,0.230523,0.230523
Other,0.230523,0.230523,0.230523,0.230523
JEEP,0.230523,0.230523,0.230523,0.230523
HYUNDAI,0.230523,0.230523,0.230523,0.230523
DODGE,0.230523,0.230523,0.230523,0.230523
SUBARU,0.230523,0.230523,0.230523,0.230523
